In [52]:
import numpy as np
import gzip

## Setup Emission, Transition, and Pi Matrices

In [77]:
# bounds for area we want to focus
lower_bound = 0
upper_bound = 500000

# error for emission probability
error = 0.01

# using ceu to lower runtime since everything will take a while/lots of space
ref_hap_file = "1000GP_Phase3_yri_chr16.hap.gz"
impute_hap_file = "ps2_impute.subset.gen.gz"
legend_file = "1000GP_Phase3_chr16.legend.gz"

# read in locations and haplotypes
ref_haplotypes = gzip.open(ref_hap_file, 'rb')
legend = gzip.open(legend_file, 'rb')
imp_haplotypes = gzip.open(impute_hap_file, 'rb')

# get all known positions and haplotypes in dataset we are imputing in
# format: chr id pos ref_allele alt_allele hom_ref het hom_alt (the last 3 will have a 1 for which is true)
known_pos = {}
obs_gts = []
for known_hap in imp_haplotypes:
    gt = known_hap.decode('utf-8').strip().split(' ')
    if int(gt[2]) < lower_bound: continue
    if int(gt[2]) > upper_bound: break
    known_pos[gt[2]] = 1
    if gt[5] == 1:
        obs_gts.append(0)
    elif gt[6] == 1:
        obs_gts.append(1)
    else:
        obs_gts.append(2)

legend_metadata = []
filtered_haplotypes = []

# skip header
legend.readline()

# remember all of the biallelic sites that are unknown
unknown_sites = []
unknown_haps = []

# filter out all indices if they aren't known
# take the pos, allele0, and allele1 of metadata
for hap_line, meta_line in zip(ref_haplotypes, legend):
    metadata = meta_line.decode('utf-8').strip().split(' ')
    if int(metadata[1]) < lower_bound: continue
    if int(metadata[1]) > upper_bound: break
    if not known_pos.get(metadata[1], False):
        if "Biallelic" in metadata[4]:
            unknown_sites.append(metadata[1])
            unknown_haps.append(metadata[1:4]+[int(i) for i in hap_line.decode('utf-8').strip().split(' ')])
        continue
    haplotype = [int(i) for i in hap_line.decode('utf-8').strip().split(' ')]
    legend_metadata.append(metadata[1:4])
    filtered_haplotypes.append(haplotype)

ref_haplotypes.close()
legend.close()
imp_haplotypes.close()

assert len(filtered_haplotypes) == len(obs_gts)

# create pairs of haplotypes
H = len(filtered_haplotypes[0])
hap_pairs = int((H*(H+1))/2)

# create dictionary that can convert from number to pair of indices
ind = 0
start = 0
convert_to_pair = {}
for pos1 in range(start, H):
    for pos2 in range(pos1, H):
        convert_to_pair[ind] = (pos1, pos2)
        ind += 1
    start += 1

assert ind == hap_pairs

# emission[x,y] = P(observed gt at col y|haplotype pair x)
emission = np.zeros((hap_pairs, len(filtered_haplotypes)))
for hap_pair in range(hap_pairs):
    for pos in range(len(filtered_haplotypes)):
        i, j = convert_to_pair[hap_pair]
        
        # grab reference genotype of the haplotype pair chosen i, j
        ref_gt = (filtered_haplotypes[pos][i] + filtered_haplotypes[pos][j])
        
        # generate emission probability
        if obs_gts[pos] == ref_gt and ref_gt == 1:
            emission[hap_pair, pos] = (1-error)**2 + error**2
        elif not (obs_gts[pos] == ref_gt) and (ref_gt == 1):
            emission[hap_pair, pos] = 2*(1-error)*error
        elif obs_gts[pos] == ref_gt and ref_gt%2==0:
            emission[hap_pair, pos] = (1-error)**2
        elif obs_gts[pos] == 1 and ref_gt%2 == 0:
            emission[hap_pair, pos] = (1-error)*error
        else:
            emission[hap_pair, pos] = error**2

            
# transition mtx where every changing pairs is slightly penalized and staying is same
# starting pi values are all equal (couldn't find actual values based on LD)
transition = np.ones((hap_pairs, hap_pairs))
for i in range(0, transition.shape[0]):
    transition[i,i] += 1
transition = transition/(hap_pairs+1)
pi = np.ones((hap_pairs, ))/hap_pairs

## Viterbi Algorithm

In [78]:
# viterbi algorithm
# to perform the algorithm act as if the only locations are the known ones and
# then when all states are found go back and determine what haplotype based on nearest SNPS
viterbi = np.zeros((emission.shape[0], emission.shape[1]))
viterbi[:,0] = np.multiply(pi, emission[:,0])

# store what state the new probability came from for path
backtrack = np.zeros((emission.shape[0], emission.shape[1]))
for t in range(1, emission.shape[1]):
    for state in range(hap_pairs):
        # all possible probabilities at a certain haplotype 
        possible_probs = emission[state,t]*transition[state,:]*viterbi[:,t-1]
        # calculate max 
        viterbi[state,t] = max(possible_probs)
        # store what state came from
        backtrack[state,t] = np.argmax(possible_probs)

# backtrack to find all optimal states
path = []
last_best_state = np.argmax(viterbi[:,-1])
path.append(last_best_state)
for t in range(emission.shape[1]-1, 0,-1):
    path.insert(0, int(backtrack[last_best_state,t]))
    last_best_state = int(backtrack[last_best_state,t])

print(path)

[2526, 2526, 2526, 2526, 2526, 1491, 1070, 1070, 1070, 1070, 858, 858, 858, 858, 858, 858, 216, 216, 216, 0, 0, 0, 1908, 1908, 1908, 1908, 1908, 1908, 216, 216, 1908, 1908, 216, 431, 2526, 2526, 645, 645, 645, 645, 216, 7146, 7146, 0, 0, 0, 0, 858, 858, 858, 858, 15308, 15308, 15308, 15308, 15308, 2526, 0, 0, 645, 645, 7683, 7683, 7683, 7683, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Recover Haplotypes

In [80]:
# guess all of the unknowns from unknown_sites = []
# from path get locations 

# find closest known site for the current unknown site
known_keys = sorted([int(key) for key in list(known_pos.keys())])
closest = 0
imputed_gts = []

for i, pos in enumerate(unknown_sites):
    if closest == len(known_keys)-1:
        # get coords of haplotype pair
        coord1, coord2 = convert_to_pair[path[closest]]
        # update offset from inserting position and ref/alt alleles
        coord1 += 3
        coord2 += 3
        # output imputed genotype
        imputed_gt = unknown_haps[i][coord1] + unknown_haps[i][coord2]
        imputed_gts.append(unknown_haps[i][:3]+[imputed_gt])
        continue
    
    current_dist = np.abs(int(pos) - known_keys[closest])
    next_dist = np.abs(int(pos) - known_keys[closest+1])
    # find closest known haplotype pair 
    while current_dist > next_dist:
        closest += 1
        if closest == len(known_keys) - 1: break
        current_dist = np.abs(int(pos) - known_keys[closest])
        next_dist = np.abs(int(pos) - known_keys[closest+1])
    # output genotype
    coord1, coord2 = convert_to_pair[path[closest]]
    # update offset from inserting position and ref/alt alleles
    coord1 += 3
    coord2 += 3
    # output imputed genotype
    imputed_gt = unknown_haps[i][coord1] + unknown_haps[i][coord2]
    imputed_gts.append(unknown_haps[i][:3]+[imputed_gt])
    
known_gts = []
for ind in range(len(legend_metadata)):
    known_gts.append(legend_metadata[ind]+[obs_gts[ind]])
all_gts = imputed_gts + known_gts
all_gts.sort(key=lambda x: int(x[0]))

# write imputed genotypes to file
output = open("./my_impute_alg_yri.txt", 'a')
output.write("chr\tpos\tref\talt\tgenotype\n")
for genotype in all_gts:
    output.write("%s\t%s\t%s\t%s\t%i\n"%("16", genotype[0], genotype[1], genotype[2], genotype[3]))
    
output.close()

## Run impute 2

In [83]:
%%bash

# Run impute 2 on the four ref panels (ceu, yri, ceu_yri, not_asw)
# Hint: you'll need to tell it where to find: 
# - genotypes (from the "subest" file, already in gen file format required for IMPUTE2)
# - tell it to only look in the interval 5e6 to 10e6
# See https://mathgen.stats.ox.ac.uk/impute/impute_v2.html for more info on how to run

# Remove the "NotImplemented" line below.
# This gets inserted automatically for Python code
# Since nbgrader doesn't know this is a bash cell

# Example impute2 command: 
# impute2 -m genetic_map_chr${chrom}_combined_b37.txt \
#     -h 1000GP_Phase3_${refpanel}_chr${chrom}.hap.gz \
#     -l 1000GP_Phase3_chr${chrom}.legend.gz \
#     -phase -Ne 20000
#     -g mygenotypes.gen
#     -int <from> <to>
#     -o ~/ps2/ps2_impute.phased.${refpanel}.impute2

DATADIR=/datasets/cs284-sp21-A00-public/ps2
chrom=16

impute2 -m ${DATADIR}/imputation/1000GP_Phase3/genetic_map_chr${chrom}_combined_b37.txt \
    -h ${DATADIR}/imputation/1000GP_Phase3/1000GP_Phase3_yri_chr${chrom}.hap.gz \
    -l ${DATADIR}/imputation/1000GP_Phase3/1000GP_Phase3_chr${chrom}.legend.gz \
    -phase -Ne 20000 \
    -g ${DATADIR}/ps2_impute.subset.gen.gz \
    -int 0 5e5 \
    -o ./ps2_impute.phased.yri.impute2


 IMPUTE version 2.3.2 

Copyright 2008 Bryan Howie, Peter Donnelly, and Jonathan Marchini
Please see the LICENCE file included with this program for conditions of use.

The seed for the random number generator is 1754299535.

Command-line input: impute2 -m /datasets/cs284-sp21-A00-public/ps2/imputation/1000GP_Phase3/genetic_map_chr16_combined_b37.txt -h /datasets/cs284-sp21-A00-public/ps2/imputation/1000GP_Phase3/1000GP_Phase3_yri_chr16.hap.gz -l /datasets/cs284-sp21-A00-public/ps2/imputation/1000GP_Phase3/1000GP_Phase3_chr16.legend.gz -phase -Ne 20000 -g /datasets/cs284-sp21-A00-public/ps2/ps2_impute.subset.gen.gz -int 0 5e5 -o ./ps2_impute.phased.yri.impute2

---------------------------------
 Nomenclature and data structure 
---------------------------------

     Panel 0: phased reference haplotypes
     Panel 2: unphased study genotypes

For optimal results, each successive panel (0,1,2) should contain a subset of the SNPs in the previous panel. When the data structure deviates f

## R Squared comparing my algorithm and impute2/beagle

In [7]:
%%bash

python3 pset2_impute_my_data.py \
  ./my_impute_alg_yri.txt \
  ./ps2_impute.heldout.gen.gz \
  ./1000GP_Phase3_chr16.legend.gz

YRI 0.3622466193358977 17234

0 nan

0.0001 nan

0.001 3.0293678022431386e-07

0.01 0.0008678433083900325

0.05 0.01804026414804819

0.1 0.017397477335801933

0.2 0.05840825620118725

0.3 0.0784191903710226

0.4 0.10843253673849403

0.5 0.13296404627425928



/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [9]:
%%bash

python3 pset2_impute2_data.py \
  ./ps2_impute.phased.yri.impute2 \
  ./ps2_impute.heldout.gen.gz \
  ./1000GP_Phase3_chr16.legend.gz

yri 0.884029481495664 17240

0 nan

0.0001 nan

0.001 0.2459042146268713

0.01 0.31978765385990243

0.05 0.6770876833917581

0.1 0.6597446162713401

0.2 0.6866287155647207

0.3 0.7227348133018558

0.4 0.7659839068948173

0.5 0.7938252329228875



/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## Table comparing Impute2 and my Algorithm

| Category |My Algorithm | Impute2 |
|----|-----|------|
| Memory | 4.3 Gb | 150 mB|
| Runtime| 2.66 Hours | 5 seconds |
| Accuracy | 0.3622466193358977 | 0.884029481495664 |
| MAF 0.001 Accuracy | 3.0293678022431386e-07 | 0.2459042146268713 |
| MAF 0.01 Accuracy | 0.0008678433083900325 | 0.31978765385990243 |
| MAF 0.05 Accuracy | 0.01804026414804819 | 0.6770876833917581 |
| MAF 0.1 Accuracy | 0.017397477335801933 | 0.6597446162713401 |
| MAF 0.2 Accuracy | 0.05840825620118725 | 0.6866287155647207 |
| MAF 0.3 Accuracy | 0.0784191903710226 | 0.7227348133018558 |
| MAF 0.4 Accuracy | 0.10843253673849403 | 0.7659839068948173 |
| MAF 0.5 Accuracy | 0.13296404627425928 | 0.7938252329228875 |

This is using 0-500kbp as the region. Note the time is with Jupyter lab running extremely slow. 17234 total SNPs genotyped